Домашная работа №2 по курсу "Продвинутое машинное обучение"

Выполнил: Денисов Артем, MADE-ML-22

In [16]:
import pickle
from pprint import pprint
from typing import List, Dict

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression
from scipy.stats import spearmanr, kendalltau

In [2]:
TRAIN_YEAR = '2019'
TEST_YEAR = '2020'

In [3]:
with open('players.pkl', 'rb') as pkl:
    players = pickle.load(pkl)

with open('tournaments.pkl', 'rb') as pkl:
    tournaments = pickle.load(pkl)

with open('results.pkl', 'rb') as pkl:
    result = pickle.load(pkl)

1. Из обучающей и тестовой выборки убрал турниры:
    где нет повопросных результатов для какой-либо команды
    где команды отвечали на разное количество вопросов
    где нет резульатов турнира

In [4]:
# Отфильтровать
def get_tournaments_id(year: str, tornaments: Dict, result: Dict) -> List:
    """
    function to filter tournaments which should be in certain year 
    and results for the tourmanetn should present and be valid
    
    input:
    year: str - year to filter tournaments
    tournaments: dict - dictionary with tournaments info
    result: dict - dictionary with tournaments result including answers and team members
    
    output:
    ids: list - list of tournaments ids
    """
    ids = []
    print(f"за {year} год не включены турниры:")
    for tournament_id in tournaments.keys():
        if tournaments[tournament_id]['dateStart'][:4] == year:
            if len(result[tournament_id]) == 0:
                print('  ', tournament_id, tournaments[tournament_id]['name'], 'нет результатов турнира') 
                continue
            try:
                a = result[tournament_id][0]['mask']
            except KeyError:
                print('  ', tournament_id, tournaments[tournament_id]['name'], 'нет поля mask в результатах')
                continue
            len_answers = []
            for team in result[tournament_id]:
                if team['mask'] is not None:
                    len_answers.append(len(team['mask']))  
            if len(len_answers) == 0:
               print('  ', tournament_id, tournaments[tournament_id]['name'], 'нет иформации об ответах команд') 
               continue 
            if min(len_answers) != max(len_answers):
                print('  ', tournament_id, tournaments[tournament_id]['name'], ' - команды отвечали на разное количество вопросов', min(len_answers), max(len_answers))
                continue
            ids.append(tournament_id)
    return ids

In [5]:
train_ids = get_tournaments_id(TRAIN_YEAR, tournaments, result)

за 2019 год не включены турниры:
   5462 Синхронный Кубок Ростова - общий зачёт  - команды отвечали на разное количество вопросов 36 108
   5494 Школьный чемпионат Украины нет результатов турнира
   5553 Славянка без раздаток. Общий зачёт  - команды отвечали на разное количество вопросов 36 234
   5554 Гран-при Славянки. Общий зачёт  - команды отвечали на разное количество вопросов 36 234
   5560 Вторая лига чемпионата Харьковской области нет иформации об ответах команд
   5561 Первая лига чемпионата Харьковской области нет иформации об ответах команд
   5562 Высшая лига чемпионата Харьковской области нет иформации об ответах команд
   5640 Октавы: Гала-турнир. Лига Наций: Прибалтика нет результатов турнира
   5664 Гран-при МГУ нет иформации об ответах команд
   5703 Школьный Синхрон-lite. Выпуск 3.4  - команды отвечали на разное количество вопросов 24 36
   5760  Балтийский Берег. Общий зачёт  - команды отвечали на разное количество вопросов 36 144
   5829 Летний кубок Армении нет рез

In [6]:
test_ids = get_tournaments_id(TEST_YEAR, tournaments, result)

за 2020 год не включены турниры:
   4628 Семь сорок нет результатов турнира
   5151 Яровой нет результатов турнира
   5619 Любовь, смерть и киберразумисты нет результатов турнира
   5670 Онлайн: Geek-ЧГК. Светлая сторона нет поля mask в результатах
   5694 Синхрон Мишеля Матвеева нет результатов турнира
   5705 Школьный Синхрон-lite. Выпуск 3.5  - команды отвечали на разное количество вопросов 24 36
   5714 Онлайн: (а)Синхрон-lite. Лига старта. Эпизод XV нет поля mask в результатах
   5715 (а)Синхрон-lite. Лига старта. Эпизод XVI нет результатов турнира
   5716 (а)Синхрон-lite. Лига старта. Эпизод XVII нет результатов турнира
   5734 Онлайн: Синхрон-lite. Выпуск XXXV нет результатов турнира
   5736 Задача трёх тел-2 нет результатов турнира
   5747 Синхрон десятилетия Белого шума нет результатов турнира
   5755 Балтийский Берег. 5 игра нет результатов турнира
   5758 Кубок городов нет результатов турнира
   5759 Кубок Чемпионов нет результатов турнира
   5800 Октавы: Гала-турнир нет рез

In [7]:
# result[train_ids[5]]

In [8]:
print(f"в обучающей выборке {len(train_ids)} турниров, в тестовой выборке {len(test_ids)} турниров")

в обучающей выборке 663 турниров, в тестовой выборке 169 турниров


2. В качестве базовой модели использовал модель Брэдли-Тэрри. Представил, что соревнование происходит между игроком и вопросом. Победа игрока - это правильный ответ, победа вопроса - неправильный ответ. 

logit(P(player > answer)) = rating_player - rating_answer

Чтобы оценить рейтинги игроков и вопросов можно использовать логистическую регрессию, рейтинги будут значениями коэффициентов модели.
Написал две вспомогательных функции, чтобы сохранить связь между id игрока или вопроса и номеров фактора в модели

In [9]:
def get_players_mapping(ids: List, result: dict) -> dict:
    """
    function to get list of players which participated in tournaments.
    this list will be used to get 
    
    input:
    ids: list - list of tournament ids 
    
    result: dict(player_id: feature_id)
    """
    players_dict = dict()
    counter = 0
    for tournament in ids:
        for team in result[tournament]:
            for team_memeber in team['teamMembers']:
                player_id = team_memeber['player']['id']
                if player_id not in players_dict.keys():
                    players_dict[player_id] = counter
                    counter += 1
    return players_dict

In [10]:
def get_answers_mapping(ids, result):
    """
    function to get dictionary {(tournament_id, answer_number): feature_id}
    input:
      ids: id of tournaments to be processed
      result:  - dictionary with results of tournaments
    output:
      answers_dict: {(tournament_id, answer_number): id, ...} ditionary with mapping answers  to features
    """
    answers_dict = dict()
    counter = 0
    for tournament in ids:
        for team in result[tournament]:
            if team['mask'] is None:
                continue
            else:
                for answer_number, answer in enumerate(team['mask']):
                    answers_dict[(tournament, answer_number)] = counter
                    counter += 1
                break
    return answers_dict

строится матрица факторов, где в каждой строке два значения 1 в соответстующем столцбе для игрока и -1 в соттветстующем столбце для вопроса.
Если ответ был правильным, в независимой переменной будет 1, если неправильный или некорректный ответ 0

In [11]:
def get_features_and_y(ids, players_mapping, answers_mapping, result):
    features_row = []
    features_column = []
    features_data = []
    
    y_row = []
    y_column = []
    y_data = []
    n_players = len(players_mapping)
    n_answers = len(answers_mapping)
    
    y_vector = []
    feature_counter = 0
    for tournament_id in ids:
        for team in result[tournament_id]:
            if team ['mask'] is None:
                continue
            for answer_id, answer in enumerate(team['mask']):
                for team_memeber in team['teamMembers']:
                    player_id = team_memeber['player']['id']
                    
                    features_row.append(feature_counter)
                    features_column.append(players_mapping[player_id])
                    features_data.append(1)                        
                    
                    features_row.append(feature_counter)
                    features_column.append(n_players + answers_mapping[(tournament_id, answer_id)])
                    features_data.append(-1)
                    
                    if answer == '1':
                        y_vector.append(1)
                    else:
                        y_vector.append(0)
    
                    feature_counter += 1
    features_matrix = csr_matrix((features_data, (features_row, features_column)))
    return features_matrix, np.array(y_vector, dtype=np.intc)

In [12]:
players_mapping = get_players_mapping(train_ids, result)
answers_mapping = get_answers_mapping(train_ids, result)
x_train, y_train = get_features_and_y(train_ids, players_mapping, answers_mapping, result)

In [13]:
len(players_mapping), len(answers_mapping)

(56950, 31002)

In [14]:
x_train.shape, y_train.shape

((16249575, 87952), (16249575,))

In [17]:
baseline_model = LogisticRegression(fit_intercept=False)
baseline_model.fit(x_train, y_train)

C:\Users\Asus\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(fit_intercept=False)

In [18]:
baseline_model.score(x_train, y_train)

0.7761333450259469

3. Для предсказания результатов турнира, предполагается, что все вопросы одинаковые и результат будет зависеть только от рейтинга участников команды. Рейтинг команды считается или как арифметическое среднее рейтингов игроков

In [37]:
def single_forecast(tournament, rating, players_mapping):
    """
    function to predict result of tournament and to score Spearman and Candle
    
    input:
      tournament_id: int
        id of tournament from test
      results: dict
        actual results of tournament to compare with
      rating:
        predicted rating of players
      players_maping: dict
        used to compare players from tournament and players from rating
        
    output:
      spearman_correlation: float, kendall_tau: float
    """
    predict = []
    true_rating = np.array([i for i in range(1, len(tournament) + 1)])
    counter = 1
    for team in tournament:
        team_score = []
        for team_member in team['teamMembers']:
            team_member_id = team_member['player']['id']
            if team_member_id in players_mapping.keys():
                team_member_rating = rating[players_mapping[team_member_id]]
            else:
                team_member_rating = 0
            team_score.append(team_member_rating)
        if len(team_score):
            predict.append([sum(team_score) / len(team_score), counter])
        else:
            predict.append([0, counter])
        counter += 1
    predict.sort(reverse=True)
    
    predicted_rating = [rating[1] for rating in predict]
    spearman_correlation = spearmanr(true_rating, np.array(predicted_rating))
    kendall_tau = kendalltau(true_rating, np.array(predicted_rating))
    
    return spearman_correlation[0], kendall_tau[0]

In [38]:
def test_predict(ids, results, rating, players_mapping):
    """
    function to predict on test period
    """
    avg_sp_score = []
    avg_cand_score = []
    for tournament_id in ids:
        if len(results[tournament_id]) > 1:
            spearman_correlation, kendall_tau = single_forecast(results[tournament_id], 
                                                 rating, 
                                                 players_mapping)
            avg_sp_score.append(spearman_correlation)
            avg_kend_score.append(kendall_tau)
    return sum(avg_sp_score) / len(avg_sp_score), sum(avg_kend_score) / len(avg_kend_score)

In [32]:
a, b = test_predict(test_ids, result, baseline_model.coef_[0], players_mapping)

In [35]:
print(f"корреляция Спирмена {a:.1%}, тау Кендалла {b:.1%}")

корреляция Спирмена 72.4%, тау Кендалла 55.8%


4. Долго провозился с багами базовой модели, не успел придумать и реализовать внятный EM алгоритм

5. Оценка уровня турнира по вопросам проходит аналогично. Для каждого из турниров берется средний рейтинг его вопросов

In [49]:
def get_tournaments_rating(ids, rating, result, answers_mapping):
    final_rating = []
    for tournament_id in ids:
        answers_rating = []
        for team in result[tournament_id]:
            if team['mask'] is None:
                continue
            else:
                for answer_number, answer in enumerate(team['mask']):
                    answer_id = answers_mapping[(tournament_id, answer_number)] 
                    answers_rating.append(rating[answer_id])
                break
        rating_to_add = sum(answers_rating) / len(answers_rating)
        final_rating.append([rating_to_add, tournament_id])
    return final_rating

In [56]:
tournament_rating = get_tournaments_rating(train_ids,
                                           baseline_model.coef_[0][len(players_mapping):],
                                           result, 
                                           answers_mapping)
tournament_rating.sort(reverse=True)

In [57]:
print("5 турниров с самыми сложными вопросами:")
for pair in tournament_rating[:5]:
    print('    ', tournaments[pair[1]]['name'])

5 турниров с самыми сложными вопросами
     Чемпионат Санкт-Петербурга. Первая лига
     Угрюмый Ёрш
     Синхрон высшей лиги Москвы
     Первенство правого полушария
     Воображаемый музей


In [58]:
print("5 турниров с самыми легкими вопросами:")
for pair in tournament_rating[-5:]:
    print('    ', tournaments[pair[1]]['name'])

5 турниров с самыми легкими вопросами
     (а)Синхрон-lite. Лига старта. Эпизод VII
     (а)Синхрон-lite. Лига старта. Эпизод IX
     (а)Синхрон-lite. Лига старта. Эпизод III
     Синхрон Лиги Разума
     (а)Синхрон-lite. Лига старта. Эпизод V
